In [ ]:
import pickle
import os
import pandas as pd
import numpy as np
np.random.seed(42)
from sklearn.model_selection import train_test_split as splt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Conv1D, MaxPooling1D, Flatten
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
dfy = pd.read_csv("MILANO_wSCORE.csv")
dfx = pd.read_csv("PLIC-milano-processed.csv", sep=";")

In [ ]:
y = dfy['SCORE'].values

dfx = dfx.replace(-1, np.nan).replace("-1", np.nan).replace(-1.0, np.nan)
dfx = dfx.loc[:, (dfx != 0).any(axis=0)]
dfx = dfx.dropna(how='any', axis=1)\
dfx = dfx.select_dtypes(exclude=['object'])
dfx = dfx.drop(labels = ['smoking', 'smoking recod', 'glucose', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1) #
X = dfx.values

In [ ]:
bina = 0
for i in range(X.shape[1]):
    if (np.amax(X[:,i]) == 1):
        bina += 1
print(bina)

In [ ]:
model = Sequential()
model.add(Reshape((X.shape[1],1), input_shape = (X.shape[1],)))
model.add(Conv1D(16, 4, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Conv1D(4, 2, activation = 'relu'))
#model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(4, activation='softmax'))

In [ ]:
dfx1.columns

In [ ]:
# 0   0.04   0.12   0.24   1
model.summary()
X = X/X.max(axis=0)
yy = y/50
yy = np.zeros((yy.shape[0], 4))
for i,yyy in enumerate(y):
    if(yyy < 0.01): # Low
        yy[i][0] = 1
    elif(yyy < 0.02): # Medium
        yy[i][1] = 1
    elif(yyy < 0.05): # High
        yy[i][2] = 1
    else: # Very High (aka Dead)
        yy[i][3] = 1 

In [ ]:
print(np.amax(X))
print(np.amax(y))
X_tr, X_ts, y_tr, y_ts = splt(X, yy, test_size=0.25, random_state=42)
es = EarlyStopping(patience = 100, restore_best_weights = True)

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
history = model.fit(X_tr, y_tr, validation_data=(X_ts, y_ts), epochs = 100, callbacks = [es]) #

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
model.save("model_CC_CNN_ADAM_Classifier_DS2_NewThreshold_2000.h5")

In [ ]:
pickle.dump(history, open("history_CC_CNN_ADAM_Classifier_DS2_NewThreshold_2000.pkl", "wb"))